# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.00,-57.56,75.20,64,40,18.34,AR,1607447877
1,1,bredasdorp,-34.53,20.04,68.00,77,16,14.99,ZA,1607448099
2,2,anadyr,64.75,177.48,3.20,91,40,4.47,RU,1607448099
3,3,fortuna foothills,32.66,-114.41,71.01,10,1,8.05,US,1607448099
4,4,padang,-0.95,100.35,77.83,79,100,3.85,ID,1607448099


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=300,point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                  & (city_data_df["Wind Speed"] < 10) \
                                  & (city_data_df["Cloudiness"]==0)].dropna()
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
239,239,salalah,17.02,54.09,78.80,74,0,2.24,OM,1607448133
341,341,poum,-20.23,164.02,79.77,79,0,9.89,NC,1607448148
463,463,cidreira,-30.18,-50.21,78.57,61,0,9.71,BR,1607448166
531,531,yanam,16.73,82.22,75.20,83,0,5.82,IN,1607448135


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
239,salalah,OM,17.02,54.09,
341,poum,NC,-20.23,164.02,
463,cidreira,BR,-30.18,-50.21,
531,yanam,IN,16.73,82.22,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 239: salalah.
Closest hotel in salalah is Al Dyafa Hotel Suites.
------------
Retrieving Results for Index 341: poum.
Missing field/result... skipping.
------------
Retrieving Results for Index 463: cidreira.
Closest hotel in cidreira is Hotel Castelo.
------------
Retrieving Results for Index 531: yanam.
Closest hotel in yanam is The Regency Hotel.
------------
-------End of Search-------


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
239,salalah,OM,17.02,54.09,Al Dyafa Hotel Suites
341,poum,NC,-20.23,164.02,
463,cidreira,BR,-30.18,-50.21,Hotel Castelo
531,yanam,IN,16.73,82.22,The Regency Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure

fig

Figure(layout=FigureLayout(height='420px'))